In [2]:
import glob
import os
import gc
import pandas as pd
import geopy.distance
pd.set_option('display.max_columns', None)  
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

def reduce_memory(df):
    for c in df.columns:
        if df[c].dtype=='int':
            if df[c].min()<0:
                if df[c].abs().max()<2**7:
                    df[c] = df[c].astype('int8')
                elif df[c].abs().max()<2**15:
                    df[c] = df[c].astype('int16')
                elif df[c].abs().max()<2**31:
                    df[c] = df[c].astype('int32')
                else:
                    continue
            else:
                if df[c].max()<2**8:
                    df[c] = df[c].astype('uint8')
                elif df[c].max()<2**16:
                    df[c] = df[c].astype('uint16')
                elif df[c].max()<2**32:
                    df[c] = df[c].astype('uint32')
                else:
                    continue
    return df

In [3]:
# A loop to display each dataset's schema 
for files in glob.glob("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\*.csv"):
    df = pd.read_csv(files)
    print (files, 'has ', len(df.columns), 'columns\n')
    print (df.columns, '\n')
    print (df.dtypes, '\n')

D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2011(geocoded).csv has  13 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'full_address',
       'Latitude', 'Longitude', 'storey_range', 'floor_area_sqm', 'flat_model',
       'lease_commence_date', 'resale_price'],
      dtype='object') 

month                   object
town                    object
flat_type               object
block                   object
street_name             object
full_address            object
Latitude               float64
Longitude              float64
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
dtype: object 

D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012 Jan Feb.csv has  13 columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'full_address',
       'Latitude', 'Longitude', 'storey_range', 'floor_area_sqm', 

In [4]:
flat_coord = pd.DataFrame()
for files in glob.glob("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\*.csv"):
    print ('Concatenating...', files)
    flat_coord = pd.concat([flat_coord,pd.read_csv(files).iloc[:,:]],axis=0)
print ('Datasets are now concatenated into one dataframe.')


Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2011(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012 Jan Feb.csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2012-2014(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2015-2016(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2017(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2018(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2019(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2020(geocoded).csv
Concatenating... D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Resale flat prices\Geocoded\2021(geocoded).csv
Datasets are now concatenated into one

In [5]:
flat_coord.shape


(183905, 14)

In [6]:
flat_coord=flat_coord[["full_address","Latitude","Longitude","month","town","flat_type","block","street_name","storey_range","floor_area_sqm","flat_model","lease_commence_date","resale_price","remaining_lease"]]

In [7]:
from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [8]:
mrt_coord = pd.read_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\MRT_coordinates.csv")
mrt_coord = mrt_coord[['STN_NAME','Latitude','Longitude']]
mrt_coord.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


In [9]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

,flat,mrt,mrt_dist,num_mrt_2km
0,"45 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.153912,4
1,"106 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.962478,6
2,"50 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.260606,5
3,"108 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.988774,6
4,"110 SPOTTISWOODE PK RD,Singapore",OUTRAM PARK MRT STATION,0.759241,11


In [10]:
flat_mrt

,flat,mrt,mrt_dist,num_mrt_2km
0,"45 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.153912,4
1,"106 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.962478,6
2,"50 TELOK BLANGAH DR,Singapore",TELOK BLANGAH MRT STATION,0.260606,5
3,"108 BT PURMEI RD,Singapore",HARBOURFRONT MRT STATION,0.988774,6
4,"110 SPOTTISWOODE PK RD,Singapore",OUTRAM PARK MRT STATION,0.759241,11
...,...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",SUMANG LRT STATION,0.0542452,18
16135,"512A YISHUN ST 51,Singapore",KHATIB MRT STATION,0.991286,2
16136,"506A YISHUN AVE 4,Singapore",KHATIB MRT STATION,0.815135,2
16137,"83 COMMMONWEALTH CLOSE,Singapore",COMMONWEALTH MRT STATION,0.355395,7


In [13]:
flat_mrt.to_csv(r"D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\flat_mrt.csv")

In [11]:
flat_coord

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN
2,"106 BT PURMEI RD,Singapore",1.273075,103.825657,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN
4,"108 BT PURMEI RD,Singapore",1.273442,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12421,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years
12422,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years
12423,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months
12424,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months


In [12]:
flat_mrt.rename(columns = {'flat': 'full_address'},inplace=True)

In [13]:
merged_flat_mrt=pd.merge(flat_coord, flat_mrt, on='full_address', how='left')

In [14]:
school_coord = pd.read_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\school_coordinates.csv")
school_coord.head(10)

,address,LATITUDE,LONGITUDE
0,Princess Elizabeth Primary School,1.349195,103.741000
1,Woodgrove Primary School,1.433058,103.790390
2,Nanyang Primary School,1.321115,103.806468
3,Nanyang Primary School,1.321246,103.807795
4,Waterway Primary School,1.399154,103.918671
5,Alexandra Primary School,1.291299,103.823941
6,Maris Stella High School,1.342184,103.878096
7,Maris Stella High School,1.341303,103.877814
8,Bukit View Primary School,1.345945,103.753589
9,Woodlands Ring Primary School,1.434770,103.797841


In [15]:
school_coord.rename(columns = {'address': 'school'},inplace=True)

In [16]:
school_coord

,school,LATITUDE,LONGITUDE
0,Princess Elizabeth Primary School,1.349195,103.741000
1,Woodgrove Primary School,1.433058,103.790390
2,Nanyang Primary School,1.321115,103.806468
3,Nanyang Primary School,1.321246,103.807795
4,Waterway Primary School,1.399154,103.918671
...,...,...,...
176,Pei Hwa Presbyterian Primary School,1.338067,103.776192
177,Rulang Primary School,1.346844,103.719010
178,Greenwood Primary School,1.440110,103.804629
179,Oasis Primary School,1.404409,103.910996


In [17]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'full_address', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

,full_address,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
1,"106 BT PURMEI RD,Singapore",Radin Mas Primary School,0.265618,7
2,"50 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.372768,5
3,"108 BT PURMEI RD,Singapore",Radin Mas Primary School,0.221545,8
4,"110 SPOTTISWOODE PK RD,Singapore",Cantonment Primary School,0.438143,5


In [19]:
flat_school

,full_address,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
1,"106 BT PURMEI RD,Singapore",Radin Mas Primary School,0.265618,7
2,"50 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.372768,5
3,"108 BT PURMEI RD,Singapore",Radin Mas Primary School,0.221545,8
4,"110 SPOTTISWOODE PK RD,Singapore",Cantonment Primary School,0.438143,5
...,...,...,...,...
16134,"316A PUNGGOL WAY,Singapore",Valour Primary School,0.232633,11
16135,"512A YISHUN ST 51,Singapore",Naval Base Primary School,0.360785,6
16136,"506A YISHUN AVE 4,Singapore",Naval Base Primary School,0.156402,7
16137,"83 COMMMONWEALTH CLOSE,Singapore",New Town Primary School,0.554578,5


In [ ]:
flat_coord

In [26]:
merged_flat_mrt_school=pd.merge(flat_coord, flat_school, on='full_address', how='left')

In [27]:
merged_flat_mrt_school

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,school,school_dist,num_school_2km
0,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,10 TO 12,45.0,Improved,1976,275000.0,NaN,Blangah Rise Primary School,0.454797,5
1,"45 TELOK BLANGAH DR,Singapore",1.272139,103.809627,2011-01,BUKIT MERAH,2 ROOM,45,TELOK BLANGAH DR,13 TO 15,54.0,Improved,1976,285000.0,NaN,Blangah Rise Primary School,0.454797,5
2,"106 BT PURMEI RD,Singapore",1.273075,103.825657,2011-01,BUKIT MERAH,4 ROOM,106,BT PURMEI RD,10 TO 12,104.0,Model A,1984,509000.0,NaN,Radin Mas Primary School,0.265618,7
3,"50 TELOK BLANGAH DR,Singapore",1.273087,103.810080,2011-01,BUKIT MERAH,4 ROOM,50,TELOK BLANGAH DR,07 TO 09,98.0,New Generation,1978,526000.0,NaN,Blangah Rise Primary School,0.372768,5
4,"108 BT PURMEI RD,Singapore",1.273442,103.825477,2011-01,BUKIT MERAH,4 ROOM,108,BT PURMEI RD,04 TO 06,104.0,Model A,1984,540000.0,NaN,Radin Mas Primary School,0.221545,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183900,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,04 TO 06,92.0,Model A,2015,408000.0,93 years,Northoaks Primary School,0.0739577,6
183901,"463B SEMBAWANG DR,Singapore",1.456018,103.813770,2021-06,SEMBAWANG,4 ROOM,463B,SEMBAWANG DR,19 TO 21,92.0,Model A,2015,460000.0,93 years,Northoaks Primary School,0.0739577,6
183902,"82 COMMMONWEALTH CLOSE,Singapore",1.304288,103.800480,2021-06,QUEENSTOWN,3 ROOM,82,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,265000.0,44 years 08 months,New Town Primary School,0.498291,4
183903,"83 COMMMONWEALTH CLOSE,Singapore",1.304800,103.800480,2021-06,QUEENSTOWN,3 ROOM,83,COMMMONWEALTH CLOSE,01 TO 03,60.0,Standard,1967,290000.0,44 years 08 months,New Town Primary School,0.554578,5


In [25]:
#test[test["school"].isna()==False]

In [29]:
shop_coord = pd.read_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\shoppingmall_coordinates_clean.csv")
shop_coord = shop_coord[["address",'LATITUDE','LONGITUDE']]
shop_coord.head()

,address,LATITUDE,LONGITUDE
0,Katong Square,1.304853,103.904574
1,PLQ Mall,1.317482,103.892832
2,Capitol Piazza,1.293063,103.851294
3,Bugis Cube,1.298195,103.855656
4,HillV2,1.363551,103.764236


In [30]:
school_coord.rename(columns = {'address': 'shopping_mall'},inplace=True)

In [32]:
nearest_mall = find_nearest(flat_coord, school_coord)
flat_mall = pd.DataFrame.from_dict(nearest_mall).T
flat_mall = flat_mall.rename(columns={0: 'full_address', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
flat_mall.head()

,full_address,mall,mall_dist,num_mall_2km
0,"45 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.454797,5
1,"106 BT PURMEI RD,Singapore",Radin Mas Primary School,0.265618,7
2,"50 TELOK BLANGAH DR,Singapore",Blangah Rise Primary School,0.372768,5
3,"108 BT PURMEI RD,Singapore",Radin Mas Primary School,0.221545,8
4,"110 SPOTTISWOODE PK RD,Singapore",Cantonment Primary School,0.438143,5


In [33]:
merged_flat_mrt_school_mall=pd.merge(flat_coord, flat_mall, on='full_address', how='left')

In [38]:
merged_flat_mrt_school_mall[merged_flat_mrt_school_mall["mall"].isna()==True]

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,mall,mall_dist,num_mall_2km


In [40]:
merged_flat_mrt_school_mall.to_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\merged_flat_mrt_school_mal.csv")

In [43]:
hawker_coord = pd.read_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\hawker_coordinates_clean.csv")
hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
hawker_coord.head()

,address,LATITUDE,LONGITUDE
0,Amoy Street Food Centre,1.279340,103.846653
1,Blks 20/21 Marsiling Lane,1.443444,103.776989
2,Blks 20/21 Marsiling Lane,1.443832,103.777210
3,Geylang Serai Market,1.316725,103.897977
4,Blks 2 & 3 Changi Village Road,1.443444,103.776989


In [44]:
hawker_coord.rename(columns = {'address': 'hawker'},inplace=True)

In [45]:
nearest_hawker = find_nearest(flat_coord, hawker_coord)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'full_address', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head()

,full_address,hawker,hawker_dist,num_hawker_2km
0,"45 TELOK BLANGAH DR,Singapore",Blk 11 Telok Blangah Crescent,1.1579,3
1,"106 BT PURMEI RD,Singapore",Blk 112 Jalan Bukit Merah,0.767276,6
2,"50 TELOK BLANGAH DR,Singapore",Blk 11 Telok Blangah Crescent,1.06387,5
3,"108 BT PURMEI RD,Singapore",Blk 112 Jalan Bukit Merah,0.728191,6
4,"110 SPOTTISWOODE PK RD,Singapore",Blk 6 Tanjong Pagar Plaza,0.817031,11


In [46]:
merged_flat_mrt_school_mall_hawker=pd.merge(flat_coord, flat_hawker, on='full_address', how='left')

In [48]:
merged_flat_mrt_school_mall_hawker[merged_flat_mrt_school_mall_hawker["hawker"].isna()==True]

,full_address,Latitude,Longitude,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,hawker,hawker_dist,num_hawker_2km


In [53]:
merged_flat_mrt_school_mall_hawker.to_csv("D:\BAP DATASET\HDB Resale\HDB RESALE DATA\Amenities Coordinates\merged_flat_mrt_school_mall_hawker.csv")